In [ ]:
# Motion-Only Model Evaluation Pipeline
import os
import pickle
from pathlib import Path

import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import balanced_accuracy_score, f1_score, confusion_matrix

# Configuration
PROJECT_ROOT = Path.cwd()
DATA_DIR      = PROJECT_ROOT / '../Data/Experiment_Data/2. PreprocessDataset'
TFLITE_DIR    = PROJECT_ROOT / '../Models/tflite_model/Motion'
TF_MODEL_DIR  = PROJECT_ROOT / '../Models/tensorflow_model/Motion'
OUTPUT_ACC    = PROJECT_ROOT / '../Result/Experiment_Result/Model_Accuracy'
OUTPUT_PRED   = PROJECT_ROOT / '../Result/Experiment_Result/Model_Preds'
OUTPUT_CM     = PROJECT_ROOT / '../Result/Experiment_Result/Confusion_Matrix'
NORM_PATH     = PROJECT_ROOT / f'../Normalization_params/normalization_params_motion_Right_ver6.pkl'
LB_PATH       = PROJECT_ROOT / '../LabelBinarizer/Multimodal/Label_binarizer_6_classes.pkl'

# Model names
TFLITE_MODEL  = TFLITE_DIR / 'Motion_ver6.tflite'
TF_MODEL_FILE = TF_MODEL_DIR / 'Motion_ver6/Right/Motion_Scratch.h5'

# Classes
def load_classes():
    return ['Shower','Tooth_brushing','Washing_hands','Wiping','Vacuum_Cleaner','Other']

# Load resources
def load_norm():
    return pickle.load(open(NORM_PATH,'rb'))

def load_lb():
    return pickle.load(open(LB_PATH,'rb'))

# Frame utility not needed here (data pre-windowed)

# TFLite single-sample evaluation
def predict_tflite_motion(pid: str, norm_params, lb):
    interp = tf.lite.Interpreter(model_path=str(TFLITE_MODEL))
    interp.allocate_tensors()
    input_idx = interp.get_input_details()[0]['index']
    output_idx = interp.get_output_details()[0]['index']

    # Load data
    pkl_path = DATA_DIR / pid / f'{pid}_preprocessing.pkl'
    data = pickle.load(open(pkl_path,'rb'))
    imu = data['IMU'].astype(np.float32)
    y_true = np.array(data['Activity'])

    # Normalize IMU
    pm,pn,mu,sd = [norm_params[k].reshape(1,1,-1).astype(np.float32) for k in ('max','min','mean','std')]
    imu = 1 + (imu - pm)*2/(pm-pn)
    imu = (imu - mu)/sd

    preds = []
    for sample in imu:
        inp = sample[None,...]
        interp.set_tensor(input_idx, inp)
        interp.invoke()
        out = interp.get_tensor(output_idx)[0]
        preds.append(out)

    df = pd.DataFrame(preds, columns=lb.classes_)
    df['y_true'] = y_true
    df['y_pred'] = df.drop(columns=['y_true']).idxmax(axis=1)
    return df

# TF-Keras batch evaluation
def predict_tf_motion(pid: str, norm_params, lb, batch_size=256):
    model = tf.keras.models.load_model(str(TF_MODEL_FILE))
    pkl_path = DATA_DIR / pid / f'{pid}_preprocessing_1hour.pkl'
    data = pickle.load(open(pkl_path,'rb'))
    imu = data['IMU'].astype(np.float32)
    y_true = np.array(data['Activity'])

    pm,pn,mu,sd = [norm_params[k].reshape(1,1,-1).astype(np.float32) for k in ('max','min','mean','std')]
    imu = 1 + (imu - pm)*2/(pm-pn)
    imu = (imu - mu)/sd

    preds = model.predict(imu, batch_size=batch_size)
    df = pd.DataFrame(preds, columns=lb.classes_)
    df['y_true'] = y_true
    df['y_pred'] = df.drop(columns=['y_true']).idxmax(axis=1)
    return df

# Evaluate and save
if __name__ == '__main__':
    norm = load_norm()
    lb   = load_lb()
    classes = load_classes()

    OUTPUT_ACC.mkdir(parents=True, exist_ok=True)
    OUTPUT_PRED.mkdir(parents=True, exist_ok=True)
    OUTPUT_CM.mkdir(parents=True, exist_ok=True)

    for pid in sorted(d.name for d in DATA_DIR.iterdir() if d.is_dir()):
        print(f'Evaluating motion model for participant {pid}')

        # TFLite
        df_tfl = predict_tflite_motion(pid, norm, lb)
        ba_t   = balanced_accuracy_score(df_tfl['y_true'], df_tfl['y_pred'])
        f1_t   = f1_score(df_tfl['y_true'], df_tfl['y_pred'], average='weighted')
        save_dir = OUTPUT_PRED / pid
        save_dir.mkdir(exist_ok=True)
        df_tfl.to_csv(save_dir/f'{pid}_tflite_motion.csv', index=False)
        with open(OUTPUT_ACC/f'{pid}_tflite_motion.txt','w') as f:
            f.write(f'BA: {ba_t:.4f}\nF1: {f1_t:.4f}')

        # Confusion matrix
        cm = confusion_matrix(df_tfl['y_true'], df_tfl['y_pred'], labels=classes)
        cm_pct = cm.astype(float)/cm.sum(axis=1)[:,None]*100
        fig, ax = plt.subplots(figsize=(8,6))
        im = ax.imshow(cm_pct, cmap='Blues', vmin=0, vmax=100)
        ticks = range(len(classes))
        ax.set_xticks(ticks); ax.set_xticklabels(classes, rotation=45)
        ax.set_yticks(ticks); ax.set_yticklabels(classes)
        thresh = cm_pct.max()/2
        for i in ticks:
            for j in ticks:
                ax.text(j,i,f'{cm_pct[i,j]:.1f}', ha='center',
                        color='white' if cm_pct[i,j]>thresh else 'black')
        plt.tight_layout()
        (OUTPUT_CM/ pid / 'tflite').mkdir(exist_ok=True, parents=True)
        fig.savefig(OUTPUT_CM/pid/'tflite'/f'{pid}_cm.png')
        plt.close(fig)

        # TF-Keras
        df_tf = predict_tf_motion(pid, norm, lb)
        ba_tf = balanced_accuracy_score(df_tf['y_true'], df_tf['y_pred'])
        f1_tf = f1_score(df_tf['y_true'], df_tf['y_pred'], average='weighted')
        df_tf.to_csv(save_dir/f'{pid}_tf_motion.csv', index=False)
        with open(OUTPUT_ACC/f'{pid}_tf_motion.txt','w') as f:
            f.write(f'BA: {ba_tf:.4f}\nF1: {f1_tf:.4f}')

        print(f"{pid}: TFLite BA={ba_t:.3f}, TF BA={ba_tf:.3f}")